In [ ]:
#@title Install and Auto restart
requirements_content = """accelerate==0.25.0
huggingface-hub==0.20.1
Pillow==10.1.0
transformers==4.36.2
einops==0.7.0
gradio==4.15.0
timm==0.9.12
pydub==0.25.1
edge-tts
torch==2.1.2
torchvision==0.16.2
"""
# Write the contents to the file
with open("/content/requirements.txt", "w") as file:
    file.write(requirements_content)

print(f"File '/content/requirements.txt' has been successfully created with the specified contents.")

!pip install -r /content/requirements.txt
from IPython.display import clear_output
clear_output()
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [ ]:
!nvidia-smi

Tue Jul 30 20:43:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title Downlod and Config MoonDream
import torch
import re
import time
# from moondream.moondream import detect_device, LATEST_REVISION
from threading import Thread
from transformers import TextIteratorStreamer, AutoTokenizer, AutoModelForCausalLM
from PIL import Image


import torch
if torch.cuda.is_available():
    use_cpu=False
else:
    use_cpu=True
# Specify whether to use CPU or GPU
# use_cpu = True  # Change to True if you want to use CPU

if use_cpu:
  device = torch.device("cpu")
  dtype = torch.float32
else:
  device=torch.device("cuda")
  dtype=torch.float16
print("Using device:", device)

revision = "2024-05-20"
model_id = "vikhyatk/moondream2"

tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

moondream = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
).to(device=device, dtype=dtype)
moondream.eval()
from IPython.display import clear_output
clear_output()

In [ ]:
#@title  MoonDream functions
def answer_question(image, prompt):
    image_embeds = moondream.encode_image(image)
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)
    thread = Thread(
        target=moondream.answer_question,
        kwargs={
            "image_embeds": image_embeds,
            "question": prompt,
            "tokenizer": tokenizer,
            "streamer": streamer,
        },
    )
    thread.start()

    buffer = ""
    for new_text in streamer:
        clean_text = re.sub("<$|END$", "", new_text)
        buffer += clean_text
        yield buffer.strip("<END")
def get_answer(prompt,image):
  answer = []
  for text in answer_question(image, prompt):
      answer.append(text)

  if len(answer) == 0:
      answer.append("Nothing Found")

  return answer[-1]

In [ ]:
#@title <-- Just run the cell (config edge TTS)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def chunks_sentences(paragraph, join_limit=2):
    sentences = sent_tokenize(paragraph)
    # Initialize an empty list to store the new sentences
    new_sentences = []

    # Iterate through the list of sentences in steps of 'join_limit'
    for i in range(0, len(sentences), join_limit):
        # Join the sentences with a space between them
        new_sentence = ' '.join(sentences[i:i + join_limit])
        new_sentences.append(new_sentence)
    return new_sentences


def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      filtered_list=chunks_sentences(input_text, join_limit=2)
      # temp_list = input_text.strip().split(".")
      # filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      # if temp_list[-1].strip():
      #     filtered_list.append(temp_list[-1].strip())
      return filtered_list




import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path


text = "This is edge text to speech"  # @param {type: "string"}
Language = "English" # @param ['English']

Gender = "Male"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
long_sentence = True # @param {type:"boolean"}
save_path = '/content/edge.wav'  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global long_sentence,translate_text_flag,Language,speed,voice_name
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['This is edge text to speech']
edge-tts  --rate=+0% --voice en-US-ChristopherNeural --text "This is edge text to speech" --write-media /content/audio/479bd782.mp3


In [19]:
#@title video analyze utils
import os
import shutil
import subprocess
from PIL import Image

base_path = "/content"
frames_save_at = f"{base_path}/frames"

def format_timestamp(seconds):
    """Format seconds as HH:MM:SS."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{secs:02}"

def extract_frame(video_path):
    global base_path, frames_save_at

    # Clean up and create the frames directory
    if os.path.exists(frames_save_at):
        shutil.rmtree(frames_save_at)
    os.mkdir(frames_save_at)

    # Extract frames using numbering format
    extract_frames_command = (
        f"ffmpeg -i {video_path} -vf fps=1 {frames_save_at}/%04d.png"
    )

    # Execute the ffmpeg command
    var = os.system(extract_frames_command)
    if var != 0:
        print("Error extracting frames.")
        print(f"!{extract_frames_command}")
        return

    # Get the video duration
    duration_command = f"ffprobe -v error -show_entries format=duration -of csv=p=0 {video_path}"
    result = subprocess.run(duration_command, shell=True, capture_output=True, text=True)
    duration = float(result.stdout.strip())

    # Calculate frame interval
    frame_interval = 1  # Extracting one frame per second
    total_frames = int(duration // frame_interval)

    # Create a dictionary to map timestamps to file paths
    frame_dict = {}
    for frame_number in range(total_frames):
        # Calculate the timestamp for the frame
        timestamp = format_timestamp(frame_number)  # Frame number as seconds
        frame_file = f"{frames_save_at}/{str(frame_number + 1).zfill(4)}.png"
        frame_dict[timestamp] = frame_file
    sorted_frame_dict = dict(sorted(frame_dict.items(), key=lambda item: int(os.path.basename(item[1]).split('.')[0])))
    return sorted_frame_dict,duration

import os
from pydub import AudioSegment
def process_video(input_video_path, input_audio_path, video_duration):
    # Ensure output directory exists
    output_dir = "/content/video_save"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Extract file name and generate output path
    base_name = os.path.basename(input_video_path)
    file_name, _ = os.path.splitext(base_name)
    output_video_path = os.path.join(output_dir, f"voice_over_{file_name}.mp4")
    shutil.copy(input_audio_path,f"{output_dir}/voice_over_{file_name}.wav")
    # Load audio
    audio = AudioSegment.from_file(input_audio_path)

    # Adjust the audio duration to match the given video duration
    audio_duration = len(audio) / 1000.0  # pydub audio duration in seconds
    if audio_duration < video_duration:
        # Add silence if audio is shorter
        silence_duration = (video_duration - audio_duration) * 1000  # in milliseconds
        silence =AudioSegment.silent(duration=silence_duration)
        audio = audio + silence
    elif audio_duration > video_duration:
        # Trim audio if it is longer
        audio = audio[:int(video_duration * 1000)]  # trim audio to match video duration

    # Save adjusted audio to a temporary file
    temp_audio_path = '/content/temp_audio.wav'
    audio.export(temp_audio_path, format='wav')

    # Use ffmpeg to replace the audio in the video
    command = f'ffmpeg -i {input_video_path} -i {temp_audio_path} -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 -shortest {output_video_path} -y'
    os.system(command)

    # Clean up temporary files
    os.remove(temp_audio_path)

    return output_video_path



def get_chatgpt_prompt(frame_dict,video_topic):
  chatgpt_prompt=f"""These are frames of {video_topic} video. Create a short voiceover script in the style of a commentator or naration and make it slightly emotional. Only include the narration.\n
Here is the frame TimeStamp and Description.\n"""
  prompt="Describe the image in short"
  if frame_dict:
      for timestamp, path in frame_dict.items():
          # print(f"Timestamp: {timestamp}, Frame Path: {path}")
          pil_image = Image.open(path)
          image_description = get_answer(prompt, pil_image)
          # print(image_description)

          # Append the description to chatgpt_prompt
          chatgpt_prompt += f"{timestamp}\n{image_description.strip()}\n"
  chatgpt_prompt+=f"""\n -- So, analyze the video, and write a overall voiceover narration without timestamp, remember the Video Duration is {video_duration} Seconds. So base on the video duration write as short as you.
  covert the response in text to speech it match the video sequence and video duration. calculate what is the time to speak your
  response base on the calculation write in short so it match exact video duration. Normally for 4 sentences which containing average 15 words. The TTS duration will average 24-30 seconds."""
  # Output the final prompt
  return chatgpt_prompt





In [ ]:
#@title Mount drive to upload video the speed is faster
from google.colab import drive
import os
drive.mount('/content/gdrive')
if not os.path.exists("/content/gdrive/MyDrive/video"):
  os.mkdir("/content/gdrive/MyDrive/video")

Mounted at /content/gdrive


#Colab video upload is slow

### "already_uploaded_in_drive"
1.   Open google drive
2.   Find a folder called "```video```" inside google drive
3.   Upload your video inside "```video```" folder in google drive
4.   Run next cell and select "alread_uploaded_in_drive"

### "upload_now"
It's slow

In [ ]:
# @title ##**Choose option** { display-mode: "form" }
choose= "already_uploaded_in_drive" #@param ['upload_now','already_uploaded_in_drive']
import os
from google.colab import drive
from IPython.display import clear_output

folder_path = "/content/gdrive/MyDrive/video"


from google.colab import files
import shutil

upload_folder = '/content/user_upload'

if not os.path.exists(upload_folder):
    os.mkdir(upload_folder)
upload_video=[]
if choose== "upload_now":
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
    upload_video.append(dst_path)
    clear_output()
    print(upload_video[-1])


if choose== "already_uploaded_in_drive":
  drive.mount('/content/gdrive')
  clear_output()
  if not os.path.exists("/content/gdrive/MyDrive/video"):
    os.mkdir("/content/gdrive/MyDrive/video")
  video_id=1
  table={}
  ids=[]
  videos=[]
  id_monitor={}
  video_folder="/content/gdrive/MyDrive/video"
  for i in os.listdir(video_folder):
    ids.append(video_id)
    videos.append(i)
    id_monitor[video_id]=i
    video_id=video_id+1
  table["file_name"]=videos
  table["file_id"]=ids
  import pandas as pd
  df = pd.DataFrame(table)
  try:
    df.set_index('file_id', inplace=True)
  except:
    pass
  print(df)
  print("\n")
  print("Note the File Id")


                         file_name
file_id                           
1        81945-577442929_small.mp4


Note the File Id


In [ ]:
# @title ##**leave blank if you selected 'upload_now'** { display-mode: "form" }

if choose == "already_uploaded_in_drive":
    video_id = '1'  # @param {type: "string"}
    if len(video_id) == 0:
        print("Enter Video ID")
    else:
        video_id = int(video_id)
        if video_id < len(id_monitor)+1:
            target_video_path = "/content/gdrive/MyDrive/video/" + id_monitor[video_id]
        else:
            print("Invalid Video ID")
            target_video_path = ''
else:
    target_video_path = upload_video[-1]

target_video_path

'/content/gdrive/MyDrive/video/81945-577442929_small.mp4'

In [ ]:
# Example usage
video_path="/content/gdrive/MyDrive/video/81945-577442929_small.mp4"# @param {type: "string"}
video_topic="skydiving" # @param {type: "string"}
frame_dict,video_duration = extract_frame(video_path)

chatgpt_prompt=get_chatgpt_prompt(frame_dict,video_topic)
chatgpt_prompt

'These are frames of skydiving video. Create a short voiceover script in the style of a commentator or naration and make it slightly emotional. Only include the narration.\n\nHere is the frame TimeStamp and Description.\n00:00:00\nTwo people are paragliding over a large body of water, with mountains in the background.\n00:00:01\nTwo people are paragliding over a large body of water, with mountains visible in the background.\n00:00:02\nTwo people are paragliding over a large body of water, with mountains visible in the background.\n00:00:03\nTwo people are paragliding over a large body of water, with mountains visible in the background.\n00:00:04\nTwo people are paragliding over a large body of water, with mountains in the background.\n00:00:05\nTwo people are paragliding over a large body of water, with one person flying higher than the other.\n00:00:06\nTwo people are paragliding over a large body of water, with one person flying higher than the other.\n00:00:07\nTwo people are paragl

In [15]:
chatgpt_response = "Above the tranquil waters and majestic mountains, two souls dance through the sky, their colorful parachutes painting a story of freedom and adventure. As they glide gracefully, one soars slightly ahead, a symbol of the thrill and beauty of shared dreams. In this moment, they are bound by a breathtaking unity, a celebration of the boundless sky."  # @param {type: "string"}
audio_path=talk(chatgpt_response)
Audio(audio_path, autoplay=False)

['Above the tranquil waters and majestic mountains, two souls dance through the sky, their colorful parachutes painting a story of freedom and adventure. As they glide gracefully, one soars slightly ahead, a symbol of the thrill and beauty of shared dreams.', 'In this moment, they are bound by a breathtaking unity, a celebration of the boundless sky.']
Above the tranquil waters and majestic mountains, two souls dance through the sky, their colorful parachutes painting a story of freedom and adventure. As they glide gracefully, one soars slightly ahead, a symbol of the thrill and beauty of shared dreams.
edge-tts  --rate=+0% --voice en-US-ChristopherNeural --text "Above the tranquil waters and majestic mountains, two souls dance through the sky, their colorful parachutes painting a story of freedom and adventure. As they glide gracefully, one soars slightly ahead, a symbol of the thrill and beauty of shared dreams." --write-media /content/edge_tts_voice/1.mp3
In this moment, they are bo

In [16]:
#@title download video
from google.colab import files
output_path = process_video(video_path, audio_path, video_duration)
print(f"Processed video saved at: {output_path}")
if os.path.exists("/content/gdrive/MyDrive/"):
  f_name=os.path.basename(output_path)
  drive_save_path=f"/content/gdrive/MyDrive/video/{f_name}"
  shutil.copy(output_path,drive_save_path)

files.download(output_path)


Processed video saved at: /content/video_save/voice_over_81945-577442929_small.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>